# Pre-processing Oregon Allocation data for WaDE upload.

Purpose:  To pre-process the Oregon data into one master file for simple DataFrame creation and extraction

Useful Links to Data:

- Data Avalaible (use 'Statewide Water Right Spatial Data with Metadata'): https://www.oregon.gov/OWRD/access_Data/Pages/Data.aspx

- POD metadata: https://arcgis.wrd.state.or.us/data/wr_pod_metadata.pdfPOD

- POU metadata: https://arcgis.wrd.state.or.us/data/wr_pou_metadata.pdfPOD

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Oregon/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Point of Diversoin Data

In [3]:
inputFile = "ORwr_v_pod_public_input.zip"
dfinPOD = pd.read_csv(inputFile, encoding = "ISO-8859-1")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "orD" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('ORwr_v_pod_public_input.zip', compression=dict(method='zip', archive_name='ORwr_v_pod_public_input.csv'), index=False)

dfinPOD = dfinPOD.drop_duplicates().reset_index(drop=True)
print(len(dfinPOD))
dfinPOD.head()

C:\Users\rjame\AppData\Local\Temp\ipykernel_27004\2584064887.py:2: DtypeWarning: Columns (12,26,27,46) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOD = pd.read_csv(inputFile, encoding = "ISO-8859-1")


189822


,OBJECTID,snp_id,app_nbr,pod_display,permit_nbr,acre_feet,acre_feet_est,agency,app_char,begin_day,begin_month,cert_nbr,claim_char,claim_nbr,decree_title,duty,end_day,end_month,feature_quality_code,last_updt_date,max_rate_acre_feet,max_rate_cfs,name_company,name_first,name_last,permit_char,pod_char,pod_display_short,pod_location_id,pod_nbr,pod_use_id,POINT_X,POINT_Y,priority_date,rate_cfs,rate_cfs_est,rec_creation_date,remarks,source,source_type,stream_name,streamcode,supplemental,technician_initials,transfer_nbr,tributary_to,use_category,use_code,use_code_description,field_51,wr_type,wris_link,geometry,WaDEUUID
0,1,21755,11987.00000,Permit: G 10961 * MI,10961.00000,NaN,0,OWRD,G,1.00000,1.00000,NaN,NaN,NaN,NaN,NaN,31.00000,12.00000,NaN,6/1/1996 0:00,NaN,0.04000,FORMOSA EXPLORATION INC.,NaN,NaN,G,NaN,G 10961,6909,1,26859,539412.54990,416705.79990,11/21/1989 0:00,0.04000,0,6/1/1996 0:00,0 G 10961 1,FORMOSA 1 ADIT ...,WE,UNN STR > MIDDLE CR,16100000000000000000.00000,0,MIGRT,NaN,CANYON CREEK,0,MI,MINING,NaN,GW,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,NaN,orD0
1,2,21755,11987.00000,Permit: G 10961 * MI,10961.00000,NaN,0,OWRD,G,1.00000,1.00000,NaN,NaN,NaN,NaN,NaN,31.00000,12.00000,NaN,6/1/1996 0:00,NaN,0.00500,FORMOSA EXPLORATION INC.,NaN,NaN,G,NaN,G 10961,6910,2,26860,539232.91670,416251.99180,11/21/1989 0:00,0.00500,0,6/1/1996 0:00,0 G 10961 2,SILVER BUTTE 1 ADIT ...,WE,UNN STR > MIDDLE CR,16100000000000000000.00000,0,MIGRT,NaN,CANYON CREEK,0,MI,MINING,NaN,GW,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,NaN,orD1
2,3,23327,13684.00000,Permit: G 12684 * MI,12684.00000,NaN,0,OWRD,G,1.00000,1.00000,NaN,NaN,NaN,NaN,NaN,31.00000,12.00000,NaN,6/1/2001 0:00,NaN,0.22000,DUTCH MINING LLC,NaN,NaN,G,NaN,G 12684,9355,1,29682,470932.79990,355915.45830,5/6/1994 0:00,0.11000,1,6/1/2001 0:00,0 G 12684 1,A WELL ...,WE,DRAIN CR > WHISKY CR,15200000000000.00000,0,MIGRT,NaN,ROGUE RIVER,0,MI,MINING,NaN,GW,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,NaN,orD2
3,4,23390,13944.00000,Permit: G 12750 * MI,12750.00000,NaN,0,OWRD,G,1.00000,1.00000,NaN,NaN,NaN,NaN,NaN,31.00000,12.00000,NaN,5/1/2001 0:00,NaN,0.67000,KNIFE RIVER CORP.,JEFF,STEYAERT,G,NaN,G 12750,9480,1,29835,1010123.96700,931993.46650,1/18/1995 0:00,0.67000,0,5/1/2001 0:00,0 G 12750 1,A WELL,WE,TROUT CR > INDIAN FORD CR,5200000000000000.00000,0,MIGRT,NaN,TROUT CREEK BASIN,0,MI,MINING,NaN,GW,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,NaN,orD3
4,5,23390,13944.00000,Permit: G 12750 * MI,12750.00000,NaN,0,OWRD,G,1.00000,1.00000,NaN,NaN,NaN,NaN,NaN,31.00000,12.00000,NaN,5/1/2001 0:00,NaN,0.11000,KNIFE RIVER CORP.,JEFF,STEYAERT,G,NaN,G 12750,9480,1,29836,1010123.96700,931993.46650,1/18/1995 0:00,0.11000,0,5/1/2001 0:00,0 G 12750 1,A WELL,WE,TROUT CR > INDIAN FORD CR,5200000000000000.00000,0,MIGRT,NaN,TROUT CREEK BASIN,0,MI,MINING,NaN,GW,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,NaN,orD4


In [4]:
# For creating WaterSourceTypeCV
WSTypeDict = {
    "ST": "Storage",
    "GW": "Groundwater",
    "SW": "Surface Water"}
def assignWaterSourceTypeCV(colrowValue):
    colrowValue = str(colrowValue).strip()
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "WaDE Unspecified"
    else:
        try:
            outList = WSTypeDict[colrowValue]
        except:
            outList = "WaDE Unspecified"

    return outList

dfinPOD['in_WaterSourceTypeCV'] = dfinPOD.apply(lambda row: assignWaterSourceTypeCV(row['wr_type']), axis=1)
dfinPOD['in_WaterSourceTypeCV'].unique()

array(['Groundwater', 'Surface Water', 'Storage'], dtype=object)

In [5]:
# For converting projection latitude.
from pyproj import Transformer, transform
transformer = Transformer.from_proj(2992, 4326)

# For converting projection latitude.
def assignLat(colrowValueLat, colrowValueLong):
    lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return lat

# For converting projection longitude.
def assignLong(colrowValueLat, colrowValueLong):
    lat, long = transformer.transform(colrowValueLat, colrowValueLong)
    return long

dfinPOD['in_Latitude'] = dfinPOD.apply(lambda row: assignLat(row['POINT_X'], row['POINT_Y']), axis=1)
dfinPOD['in_Longitude'] = dfinPOD.apply(lambda row: assignLong(row['POINT_X'], row['POINT_Y']), axis=1)

In [6]:
# For creating Site Name
def assignSiteName(colrowValueA, colrowValueB):
    if (colrowValueA == '' and colrowValueB == '') or (pd.isnull(colrowValueA) and pd.isnull(colrowValueB)):
        outList = "WaDE Unspecified"
    else:
        A = str(colrowValueA).strip()
        B = str(colrowValueB).strip()
        outList = A + "_" + B
    return outList

dfinPOD['in_SiteName'] = dfinPOD.apply(lambda row: assignSiteName(row['snp_id'], row['pod_nbr']), axis=1)

In [7]:
# For creating SiteTypeCV
STCVDict = {
"LK" : "lake",
"DR" : "drain",
"SP" : "spring",
"ST" : "stream",
"SL" : "slough",
"WW" : "waste water",
"WE" : "well",
"WR" : "winter runoff",
"SM" : "sump",
"PD" : "pond",
"RS" : "reservoir",
"DT" : "ditch",
"SE" : "sewage effluent",
"CN" : "canal"}
def assignSiteTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "WaDE Unspecified"
    else:
        String1 = colrowValue.strip()
        try:
            outList = STCVDict[String1]
        except:
            outList = "WaDE Unspecified"
    return outList

dfinPOD['in_SiteTypeCV'] = dfinPOD.apply(lambda row: assignSiteTypeCV(row['source_type']), axis=1)
dfinPOD['in_SiteTypeCV'].unique()

array(['well', 'stream', 'spring', 'reservoir', 'sump', 'waste water',
       'drain', 'winter runoff', 'lake', 'pond', 'ditch', 'canal',
       'sewage effluent', 'slough'], dtype=object)

In [8]:
# Changing datatype of used date fields. 
dfinPOD['priority_date'] = pd.to_datetime(dfinPOD['priority_date'], errors = 'coerce')
dfinPOD['priority_date'] = pd.to_datetime(dfinPOD["priority_date"].dt.strftime('%m/%d/%Y'))

In [9]:
# Creating Ownername.
# Concatenating first and last name of individual.
# Determining if company is available, split string.
# combine together for output.

import re

# first & last name funciton
def assignownerName(colrowValue1, colrowValue2):
    if colrowValue1 == '' or pd.isnull(colrowValue1):
        outList1 = ''
    else:
        outList1 = colrowValue1.strip()  # remove whitespace chars
    if colrowValue2 == '' or pd.isnull(colrowValue2):
        outList2 = ''
    else:
        outList2 = colrowValue2.strip()  # remove whitespace chars

    if outList1 == '' and outList2 == '':
        outList = ''
    elif outList1 == '':
        outList = outList2
    elif outList2 == '':
        outList = outList1
    else:
        outList = " ".join(map(str, [colrowValue1, colrowValue2]))
    return outList


# Business name and Concatenate
def assignownerNameORCompany(buisName, fName, lName):
    
    # Concatenating First and Last name together.
    frilasName = assignownerName(fName, lName)
    
    # Clearn Company Name Entry
    if buisName == "" or pd.isnull(buisName):
        outBuisString = ""
    else:
        buisName = str(buisName).strip()
        if ";" in buisName:
            xList = buisName.split(";")
            for index, item in enumerate(xList):
                if "," in item:
                    list1 = item.split(",")
                    list1.reverse()
                    xList[index] = "".join(list1)
                else:
                    xList[index] = item
            outBuisString = ",".join(xList)
        elif "," in buisName:
            xList = buisName.split(",")
            outBuisString = str(xList[0]).strip() + "," + str(xList[1]).strip()
        else:
            outBuisString = buisName
    
    #Concatenating together, create outString
    if frilasName == ""  or pd.isnull(frilasName):
        if outBuisString == ""  or pd.isnull(outBuisString):
            outString = "Unspecified"
        else:
            outString = outBuisString
    else:
        if outBuisString == ""  or pd.isnull(outBuisString):
            outString = frilasName
        else:
            outString = frilasName + ", " + outBuisString
        
    outString = outString.strip()
    outString = re.sub("[$@&.;,/\)(-]", "", outString).title().strip()
    
    return outString

dfinPOD['in_AllocationOwner'] = dfinPOD.apply(lambda row: assignownerNameORCompany(row['name_company'], row['name_first'], row['name_last']), axis=1)
dfinPOD['in_AllocationOwner'].unique()

array(['Formosa Exploration Inc', 'Dutch Mining Llc',
       'Jeff Steyaert Knife River Corp', ...,
       'Rre Montclair Terrace Holdings Llc', 'Forrest Warren',
       'Peter Stickney'], dtype=object)

In [10]:
#Determining AllocationTimeframe Start & End time for each site.

def formatDateString(inString1, inString2):
    #print(inString)
    try:
        valndf = str(int(inString1)).strip() + '/' + str(int(inString2)).strip()
    except:
        valndf = ''

    return valndf;

dfinPOD['in_AllocationTimeframeStart'] = dfinPOD.apply(lambda row: formatDateString(row['begin_month'], row['begin_day']), axis=1)
dfinPOD['in_AllocationTimeframeEnd'] = dfinPOD.apply(lambda row: formatDateString(row['end_month'], row['end_day']), axis=1)

In [11]:
#Fixing Beneficial Uses PRIMARY_PURPOSE

def fixBenUse(val):
    val = str(val).strip()
    if val == "IRRIGATION, LIVESTOCK AND DOMESTIC":
        outString = "IRRIGATION, LIVESTOCK, DOMESTIC"
    elif val == "IRRIGATION AND LIVESTOCK":
        outString = "IRRIGATION, LIVESTOCK"
    elif val == "LIVESTOCK AND WILDLIFE":
        outString = "LIVESTOCK, WILDLIFE"
    else:
        outString = val
    return outString

dfinPOD['use_code_description'] = dfinPOD.apply(lambda row: fixBenUse(row['use_code_description']), axis=1)
dfinPOD['use_code_description'].unique()

array(['MINING', 'HARVESTING OF CRANBERRIES', 'FROST PROTECTION',
       'NURSERY USES', 'TEMPERATURE CONTROL', 'CRANBERRY', 'GREENHOUSE',
       'AGRICULTURE USES', 'USE IN A MINT STILL', 'DAIRY BARN USES',
       'SUPPLEMENTAL FLOOD HARVESTING', 'DOMESTIC',
       'DOMESTIC INCLUDING LAWN AND GARDEN', 'DOMESTIC AND LIVESTOCK',
       'DOMESTIC EXPANDED', 'GROUP DOMESTIC', 'USE WITHIN A SCHOOL',
       'HUMAN CONSUMPTION', 'RESTROOM', 'HUMAN CONSUMPTION AND LIVESTOCK',
       'IRRIGATION', 'IRRIGATION, LIVESTOCK, DOMESTIC',
       'PRIMARY AND SUPPLEMENTAL IRRIGATION', 'SUPPLEMENTAL IRRIGATION',
       'IRRIGATION AND DOMESTIC', 'IRRIGATION OF CRANBERRIES',
       'IRRIGATION, LIVESTOCK', 'PRACTICABLY IRRIGABLE ACREAGE',
       'COMMERCIAL USES', 'INDUSTRIAL/MANUFACTURING USES',
       'GEO-THERMAL (HEATING & COOLING)', 'SAWMILL',
       'LOG DECK SPRINKLING', 'SHOP', 'LABORATORY', 'RECREATION',
       'CAMPSITE', 'SWIMMING', 'RAM', 'POWER DEVELOPMENT',
       'GEO-THERMAL(ENERGY PROD

In [12]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOD['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "ORwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "ORwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "ORwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOD['source']
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = dfinPOD['in_WaterSourceTypeCV']

# Site Info
df['in_CoordinateAccuracy'] = "WaDE Unspecified"
df['in_CoordinateMethodCV'] = "WaDE Unspecified"
df['in_County'] = "WaDE Unspecified"
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOD['in_Latitude']
df['in_Longitude'] = dfinPOD['in_Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfinPOD['in_SiteName']
df['in_SiteNativeID'] = "POD" + dfinPOD['pod_location_id'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfinPOD['in_SiteTypeCV']
df['in_StateCV'] = "OR"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = dfinPOD['duty']
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfinPOD['rate_cfs']
df['in_AllocationLegalStatusCV'] = "WaDE Unspecified"
df['in_AllocationNativeID'] =  dfinPOD['snp_id'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_AllocationOwner'] = dfinPOD['in_AllocationOwner']
df['in_AllocationPriorityDate'] = dfinPOD['priority_date']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = dfinPOD['in_AllocationTimeframeEnd']
df['in_AllocationTimeframeStart'] = dfinPOD['in_AllocationTimeframeStart']
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfinPOD['acre_feet']
df['in_BeneficialUseCategory'] = dfinPOD['use_code_description']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfinPOD['wris_link']  #for WaterAllocationNativeURL

outPOD = df.copy()
outPOD = outPOD.drop_duplicates().reset_index(drop=True).replace(np.nan, '')
print(len(outPOD))
outPOD.head()

189822


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,orD0,ORwr_M1,ORwr_V1,ORwr_O1,,,,FORMOSA 1 ADIT ...,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.85581,-123.38288,,,POD,21755_1,POD6909,,well,OR,,,,,,,,,,0.04000,WaDE Unspecified,21755,Formosa Exploration Inc,1989-11-21,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
1,orD1,ORwr_M1,ORwr_V1,ORwr_O1,,,,SILVER BUTTE 1 ADIT ...,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.85455,-123.38349,,,POD,21755_2,POD6910,,well,OR,,,,,,,,,,0.00500,WaDE Unspecified,21755,Formosa Exploration Inc,1989-11-21,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
2,orD2,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL ...,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.68227,-123.62942,,,POD,23327_1,POD9355,,well,OR,,,,,,,,,,0.11000,WaDE Unspecified,23327,Dutch Mining Llc,1994-05-06,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
3,orD3,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,44.30104,-121.65463,,,POD,23390_1,POD9480,,well,OR,,,,,,,,,,0.67000,WaDE Unspecified,23390,Jeff Steyaert Knife River Corp,1995-01-18,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
4,orD4,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,44.30104,-121.65463,,,POD,23390_1,POD9480,,well,OR,,,,,,,,,,0.11000,WaDE Unspecified,23390,Jeff Steyaert Knife River Corp,1995-01-18,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...


## Place of Use Data

In [13]:
inputFile = "ORwr_v_pou_public_input.zip"
dfinPOU = pd.read_csv(inputFile, encoding = "ISO-8859-1")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOU:
    dfinPOU['WaDEUUID'] = "orU" + dfinPOU.index.astype(str)
    dfinPOU.to_csv('ORwr_v_pou_public_input.zip', compression=dict(method='zip', archive_name='ORwr_v_pou_public_input.csv'), index=False)

dfinPOU = dfinPOU.drop_duplicates().reset_index(drop=True)
print(len(dfinPOU))
dfinPOU.head()

C:\Users\rjame\AppData\Local\Temp\ipykernel_27004\3711942071.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOU = pd.read_csv(inputFile, encoding = "ISO-8859-1")


109576


,OID_,snp_id,agency,app_char,app_nbr,cert_nbr,claim_char,claim_nbr,decree_title,delta_size,feature_quality_code,last_updt_date,Latitude,Longitude,name_company,name_first,name_last,permit_char,permit_nbr,pou_display,pou_display_short,pou_use_id,priority_date,rec_creation_date,remarks,Shape_Area,Shape_Length,supplemental,technician_initials,transfer_nbr,use_category,use_code,use_code_description,wr_type,wris_acres,wris_link,WaDEUUID
0,1,5135,OWRD,P,81441.00000,NaN,NaN,NaN,NaN,NaN,30.00000,7/22/2005 8:02,43.73686,-118.36448,DASH W BAR RANCH,TERRY,WILLIAMS,NaN,NaN,App: P 81441 * LV,P 81441,4124,8/7/1996 0:00,6/30/2005 0:00,PLACED USING DRG,0.00000,0.00894,0,MW,NaN,8,LV,LIVESTOCK,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU0
1,2,6333,OWRD,P,82980.00000,NaN,NaN,NaN,NaN,NaN,30.00000,4/11/2016 10:48,42.44149,-123.04144,NaN,SOFIA,PARKER,NaN,NaN,App: P 82980 * ST,P 82980,5886,1/7/1997 0:00,4/11/2016 0:00,PLACED USING 2014 IMAGERY,0.00000,0.00456,0,BRW,NaN,M,ST,STORAGE,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU1
2,3,17008,OWRD,NaN,NaN,NaN,NaN,NaN,East Mud Creek,NaN,NaN,12/1/1997 0:00,45.99292,-118.44077,NaN,DONALD,RENCKEN,NaN,NaN,Inchoate: T 4213 CF (REG) * IR,T 4213,7696,12/31/1892 0:00:00,12/1/1997 0:00,0 CD 31,0.00000,0.00819,0,MIGRT,T 4213,3,IR,IRRIGATION,SW,10.00000,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU2
3,4,17008,OWRD,NaN,NaN,NaN,NaN,NaN,East Mud Creek,NaN,NaN,12/1/1997 0:00,45.99382,-118.43963,NaN,DONALD,RENCKEN,NaN,NaN,Inchoate: T 4213 CF (REG) * LV,T 4213,7697,12/31/1892 0:00:00,12/1/1997 0:00,0 CD 31,0.00000,0.00042,0,MIGRT,T 4213,8,LV,LIVESTOCK,SW,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU3
4,5,17022,OWRD,NaN,NaN,NaN,NaN,NaN,Squaw Creek [Whychus Creek] Decree,NaN,NaN,5/1/2001 0:00,44.23531,-121.55126,NaN,ROY M,RUNCO,NaN,NaN,Inchoate: T 4784 CF (REG) * IR,T 4784,7713,12/31/1900 0:00,5/1/2001 0:00,0 CD 310,0.00002,0.01611,0,MIGRT,T 4784,3,IR,IRRIGATION,SW,38.50000,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU4


In [14]:
# POU data is missing key inputs, will combine with POD data to fill in missing gaps.
dfinPOU['snp_id'] = dfinPOU['snp_id'].astype(str)  #for AllocationNativeID

dfinPOU = pd.merge(dfinPOU, outPOD, left_on='snp_id', right_on='in_AllocationNativeID', how='left')
print(len(dfinPOU))
dfinPOU.head()

307014


,OID_,snp_id,agency,app_char,app_nbr,cert_nbr,claim_char,claim_nbr,decree_title,delta_size,feature_quality_code,last_updt_date,Latitude,Longitude,name_company,name_first,name_last,permit_char,permit_nbr,pou_display,pou_display_short,pou_use_id,priority_date,rec_creation_date,remarks,Shape_Area,Shape_Length,supplemental,technician_initials,transfer_nbr,use_category,use_code,use_code_description,wr_type,wris_acres,wris_link,WaDEUUID_x,WaDEUUID_y,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,1,5135,OWRD,P,81441.00000,NaN,NaN,NaN,NaN,NaN,30.00000,7/22/2005 8:02,43.73686,-118.36448,DASH W BAR RANCH,TERRY,WILLIAMS,NaN,NaN,App: P 81441 * LV,P 81441,4124,8/7/1996 0:00,6/30/2005 0:00,PLACED USING DRG,0.00000,0.00894,0,MW,NaN,8,LV,LIVESTOCK,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU0,orD118191,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,43.73573,-118.36361,,,POD,5135_1,POD26615,,winter runoff,OR,,,,,,,,,,,WaDE Unspecified,5135,Terry Williams Dash W Bar Ranch,1996-08-07,,12/31,1/1,,2.40000,LIVESTOCK,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
1,1,5135,OWRD,P,81441.00000,NaN,NaN,NaN,NaN,NaN,30.00000,7/22/2005 8:02,43.73686,-118.36448,DASH W BAR RANCH,TERRY,WILLIAMS,NaN,NaN,App: P 81441 * LV,P 81441,4124,8/7/1996 0:00,6/30/2005 0:00,PLACED USING DRG,0.00000,0.00894,0,MW,NaN,8,LV,LIVESTOCK,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU0,orD179500,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,43.73573,-118.36361,,,POD,5135_1,POD26615,,winter runoff,OR,,,,,,,,,,,WaDE Unspecified,5135,Terry Williams Dash W Bar Ranch,1996-08-07,,12/31,1/1,,2.40000,WILDLIFE,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
2,2,6333,OWRD,P,82980.00000,NaN,NaN,NaN,NaN,NaN,30.00000,4/11/2016 10:48,42.44149,-123.04144,NaN,SOFIA,PARKER,NaN,NaN,App: P 82980 * ST,P 82980,5886,1/7/1997 0:00,4/11/2016 0:00,PLACED USING 2014 IMAGERY,0.00000,0.00456,0,BRW,NaN,M,ST,STORAGE,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU1,orD162197,ORwr_M1,ORwr_V1,ORwr_O1,,,,A SPRING,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,42.44191,-123.04151,,,POD,6333_1,POD29464,,spring,OR,,,,,,,,,,,WaDE Unspecified,6333,Sofia Parker,1997-01-07,,12/31,1/1,,51000.00000,STORAGE,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
3,3,17008,OWRD,NaN,NaN,NaN,NaN,NaN,East Mud Creek,NaN,NaN,12/1/1997 0:00,45.99292,-118.44077,NaN,DONALD,RENCKEN,NaN,NaN,Inchoate: T 4213 CF (REG) * IR,T 4213,7696,12/31/1892 0:00:00,12/1/1997 0:00,0 CD 31,0.00000,0.00819,0,MIG

In [15]:
dfinPOU

,OID_,snp_id,agency,app_char,app_nbr,cert_nbr,claim_char,claim_nbr,decree_title,delta_size,feature_quality_code,last_updt_date,Latitude,Longitude,name_company,name_first,name_last,permit_char,permit_nbr,pou_display,pou_display_short,pou_use_id,priority_date,rec_creation_date,remarks,Shape_Area,Shape_Length,supplemental,technician_initials,transfer_nbr,use_category,use_code,use_code_description,wr_type,wris_acres,wris_link,WaDEUUID_x,WaDEUUID_y,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,1,5135,OWRD,P,81441.00000,NaN,NaN,NaN,NaN,NaN,30.00000,7/22/2005 8:02,43.73686,-118.36448,DASH W BAR RANCH,TERRY,WILLIAMS,NaN,NaN,App: P 81441 * LV,P 81441,4124,8/7/1996 0:00,6/30/2005 0:00,PLACED USING DRG,0.00000,0.00894,0,MW,NaN,8,LV,LIVESTOCK,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU0,orD118191,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,43.73573,-118.36361,,,POD,5135_1,POD26615,,winter runoff,OR,,,,,,,,,,,WaDE Unspecified,5135,Terry Williams Dash W Bar Ranch,1996-08-07,,12/31,1/1,,2.40000,LIVESTOCK,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
1,1,5135,OWRD,P,81441.00000,NaN,NaN,NaN,NaN,NaN,30.00000,7/22/2005 8:02,43.73686,-118.36448,DASH W BAR RANCH,TERRY,WILLIAMS,NaN,NaN,App: P 81441 * LV,P 81441,4124,8/7/1996 0:00,6/30/2005 0:00,PLACED USING DRG,0.00000,0.00894,0,MW,NaN,8,LV,LIVESTOCK,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU0,orD179500,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,43.73573,-118.36361,,,POD,5135_1,POD26615,,winter runoff,OR,,,,,,,,,,,WaDE Unspecified,5135,Terry Williams Dash W Bar Ranch,1996-08-07,,12/31,1/1,,2.40000,WILDLIFE,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
2,2,6333,OWRD,P,82980.00000,NaN,NaN,NaN,NaN,NaN,30.00000,4/11/2016 10:48,42.44149,-123.04144,NaN,SOFIA,PARKER,NaN,NaN,App: P 82980 * ST,P 82980,5886,1/7/1997 0:00,4/11/2016 0:00,PLACED USING 2014 IMAGERY,0.00000,0.00456,0,BRW,NaN,M,ST,STORAGE,ST,NaN,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,orU1,orD162197,ORwr_M1,ORwr_V1,ORwr_O1,,,,A SPRING,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326.00000,,,,42.44191,-123.04151,,,POD,6333_1,POD29464,,spring,OR,,,,,,,,,,,WaDE Unspecified,6333,Sofia Parker,1997-01-07,,12/31,1/1,,51000.00000,STORAGE,,,,,,0.00000,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
3,3,17008,OWRD,NaN,NaN,NaN,NaN,NaN,East Mud Creek,NaN,NaN,12/1/1997 0:00,45.99292,-118.44077,NaN,DONALD,RENCKEN,NaN,NaN,Inchoate: T 4213 CF (REG) * IR,T 4213,7696,12/31/1892 0:00:00,12/1/1997 0:00,0 CD 31,0.00000,0.00819,0,MIG

In [16]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOU['WaDEUUID_x']

# Method Info
df['in_MethodUUID'] = "ORwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "ORwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "ORwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOU['in_WaterSourceName']
df['in_WaterSourceNativeID'] = ""
df['in_WaterSourceTypeCV'] = dfinPOU['in_WaterSourceTypeCV']

# Site Info
df['in_CoordinateAccuracy'] = "WaDE Unspecified"
df['in_CoordinateMethodCV'] = "WaDE Unspecified"
df['in_County'] = "WaDE Unspecified"
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOU['Latitude']
df['in_Longitude'] = dfinPOU['Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POU"
df['in_SiteName'] = "WaDE Unspecified"
df['in_SiteNativeID'] = "POU" + dfinPOU['pou_use_id'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = "WaDE Unspecified"
df['in_StateCV'] = "OR"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = dfinPOU['in_AllocationCropDutyAmount']
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfinPOU['in_AllocationFlow_CFS']
df['in_AllocationLegalStatusCV'] = "WaDE Unspecified"
df['in_AllocationNativeID'] =  dfinPOD['snp_id'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_AllocationOwner'] = dfinPOU['in_AllocationOwner']
df['in_AllocationPriorityDate'] = dfinPOD['priority_date']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = dfinPOU['in_AllocationTimeframeEnd']
df['in_AllocationTimeframeStart'] = dfinPOU['in_AllocationTimeframeStart']
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfinPOU['in_AllocationVolume_AF']
df['in_BeneficialUseCategory'] = dfinPOU['in_BeneficialUseCategory']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = dfinPOU['wris_acres']
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfinPOU['in_WaterAllocationNativeURL']  #for WaterAllocationNativeURL

outPOU = df.copy()
outPOU = outPOU.drop_duplicates().reset_index(drop=True).replace(np.nan, '')
print(len(outPOU))
outPOU.head()

272348


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,orU0,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,43.73686,-118.36448,,,POU,WaDE Unspecified,POU4124,,WaDE Unspecified,OR,,,,,,,,,,,WaDE Unspecified,21755,Terry Williams Dash W Bar Ranch,1989-11-21,,12/31,1/1,,2.40000,LIVESTOCK,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
1,orU0,ORwr_M1,ORwr_V1,ORwr_O1,,,,RUNOFF ...,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,43.73686,-118.36448,,,POU,WaDE Unspecified,POU4124,,WaDE Unspecified,OR,,,,,,,,,,,WaDE Unspecified,21755,Terry Williams Dash W Bar Ranch,1989-11-21,,12/31,1/1,,2.40000,WILDLIFE,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
2,orU1,ORwr_M1,ORwr_V1,ORwr_O1,,,,A SPRING,,Storage,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.44149,-123.04144,,,POU,WaDE Unspecified,POU5886,,WaDE Unspecified,OR,,,,,,,,,,,WaDE Unspecified,23327,Sofia Parker,1994-05-06,,12/31,1/1,,51000.00000,STORAGE,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
3,orU2,ORwr_M1,ORwr_V1,ORwr_O1,,,,EAST BRANCH MUD CREEK ...,,Surface Water,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,45.99292,-118.44077,,,POU,WaDE Unspecified,POU7696,,WaDE Unspecified,OR,,,,,,,,,,0.13000,WaDE Unspecified,23390,Donald Rencken,1995-01-18,,12/31,1/1,,,IRRIGATION,,,,,,0,,10.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
4,orU2,ORwr_M1,ORwr_V1,ORwr_O1,,,,EAST BRANCH MUD CREEK ...,,Surface Water,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,45.99292,-118.44077,,,POU,WaDE Unspecified,POU7696,,WaDE Unspecified,OR,,,,,,,,,,0.00000,WaDE Unspecified,23390,Donald Rencken,1995-01-18,,12/31,1/1,,,LIVESTOCK,,,,,,0,,10.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...


## Concatenate POD and POU data

In [17]:
# Concatenate dataframes
frames = [outPOD, outPOU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

462170


## Custom WaDE Elements due to missing info

In [18]:
# Clean owner name up
def cleanOwnerDataFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().strip()
    return Val
outdf['in_AllocationOwner'] = outdf.apply(lambda row: cleanOwnerDataFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['Formosa Exploration Inc', 'Dutch Mining Llc',
       'Jeff Steyaert Knife River Corp', ..., 'Forrest Warren',
       'Peter Stickney', ''], dtype=object)

In [19]:
# Fixing empty string names

def fixEmptyString(val):
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        outString = val
    return outString

In [20]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: fixEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['FORMOSA 1 ADIT                                    ',
       'SILVER BUTTE 1 ADIT                               ',
       'A WELL                                            ', ...,
       'CURRIN CREEK', 'WEST FORK PALMER CREEK', 'WaDE Unspecified'],
      dtype=object)

In [21]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: fixEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['Groundwater', 'Surface Water', 'Storage', 'WaDE Unspecified'],
      dtype=object)

In [22]:
outdf['in_SiteName'] = outdf.apply(lambda row: fixEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['21755_1', '21755_2', '23327_1', ..., '200774_1', '202088_1',
       'WaDE Unspecified'], dtype=object)

In [23]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: fixEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['well', 'stream', 'spring', 'reservoir', 'sump', 'waste water',
       'drain', 'winter runoff', 'lake', 'pond', 'ditch', 'canal',
       'sewage effluent', 'slough', 'WaDE Unspecified'], dtype=object)

In [24]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['WaDE Unspecified'], dtype=object)

In [25]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['Formosa Exploration Inc', 'Dutch Mining Llc',
       'Jeff Steyaert Knife River Corp', ..., 'Forrest Warren',
       'Peter Stickney', 'WaDE Unspecified'], dtype=object)

In [26]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: fixEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['MINING', 'HARVESTING OF CRANBERRIES', 'FROST PROTECTION',
       'NURSERY USES', 'TEMPERATURE CONTROL', 'CRANBERRY', 'GREENHOUSE',
       'AGRICULTURE USES', 'USE IN A MINT STILL', 'DAIRY BARN USES',
       'SUPPLEMENTAL FLOOD HARVESTING', 'DOMESTIC',
       'DOMESTIC INCLUDING LAWN AND GARDEN', 'DOMESTIC AND LIVESTOCK',
       'DOMESTIC EXPANDED', 'GROUP DOMESTIC', 'USE WITHIN A SCHOOL',
       'HUMAN CONSUMPTION', 'RESTROOM', 'HUMAN CONSUMPTION AND LIVESTOCK',
       'IRRIGATION', 'IRRIGATION, LIVESTOCK, DOMESTIC',
       'PRIMARY AND SUPPLEMENTAL IRRIGATION', 'SUPPLEMENTAL IRRIGATION',
       'IRRIGATION AND DOMESTIC', 'IRRIGATION OF CRANBERRIES',
       'IRRIGATION, LIVESTOCK', 'PRACTICABLY IRRIGABLE ACREAGE',
       'COMMERCIAL USES', 'INDUSTRIAL/MANUFACTURING USES',
       'GEO-THERMAL (HEATING & COOLING)', 'SAWMILL',
       'LOG DECK SPRINKLING', 'SHOP', 'LABORATORY', 'RECREATION',
       'CAMPSITE', 'SWIMMING', 'RAM', 'POWER DEVELOPMENT',
       'GEO-THERMAL(ENERGY PROD

In [27]:
# in_Latitude & in_Longitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna(0)
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna(0)
outdf.head(1)

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,orD0,ORwr_M1,ORwr_V1,ORwr_O1,,,,FORMOSA 1 ADIT ...,,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.85581,-123.38288,,,POD,21755_1,POD6909,,well,OR,,,,,,,,,,0.04000,WaDE Unspecified,21755,Formosa Exploration Inc,1989-11-21,,12/31,1/1,,,MINING,,,,,,0,,,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...


In [28]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'], errors = 'coerce')
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

array(['1989-11-21T00:00:00.000000000', '1994-05-06T00:00:00.000000000',
       '1995-01-18T00:00:00.000000000', ...,
       '1993-06-11T00:00:00.000000000', '2000-01-31T00:00:00.000000000',
       '2019-07-13T00:00:00.000000000'], dtype='datetime64[ns]')

In [29]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').fillna(0)
outdf['in_AllocationFlow_CFS'].unique()

array([4.000e-02, 5.000e-03, 1.100e-01, ..., 1.052e+01, 1.581e-01,
       1.626e-01])

In [30]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').fillna(0)
outdf['in_AllocationVolume_AF'].unique()

array([ 0.  , 40.  ,  3.  , ..., 16.25, 59.1 ,  8.64])

In [31]:
# Fixing in_AllocationCropDutyAmount datatype
outdf['in_AllocationCropDutyAmount'] = pd.to_numeric(outdf['in_AllocationCropDutyAmount'], errors='coerce').fillna(0)
outdf['in_AllocationCropDutyAmount'].unique()

array([0.0000e+00, 2.5000e+00, 5.0000e+00, 3.0000e+00, 1.0000e+00,
       4.5000e+00, 5.0000e-02, 1.5000e-01, 4.0000e+00, 3.2900e+00,
       3.4000e+00, 1.6000e+00, 3.5200e+00, 1.5000e+00, 3.5000e+00,
       3.2000e+00, 1.3750e+00, 8.5500e-01, 2.4400e-01, 1.0080e+00,
       2.5800e+00, 1.8330e+00, 2.1400e-01, 6.1000e-02, 3.1000e-02,
       1.7110e+00, 1.6400e-01, 7.3300e-01, 9.2800e-01, 5.1900e-01,
       8.0000e-01, 1.9000e+00, 5.1000e-01, 2.5000e-03, 2.0000e+00,
       1.2000e+01, 7.5000e+00, 6.0000e+00, 3.9000e+00, 4.6000e+00,
       4.4000e+00, 4.8000e+00, 7.0000e+00, 1.7000e-01, 3.0000e-02,
       3.6000e+00, 6.3000e-01, 1.2000e+00, 2.6000e+00, 4.9000e+00,
       1.5000e+01, 4.2000e+01, 4.2500e+01, 2.2500e+01, 7.5000e-01,
       4.2000e+00, 5.0000e-01, 2.2300e+00, 1.3000e+00, 3.0000e+01,
       3.5800e+00, 3.2500e+00, 3.0000e-01, 9.0000e-01, 7.0000e-01,
       3.7500e+00, 1.9700e+00, 2.4000e+00, 2.6250e+00, 3.0600e+00,
       4.3000e+00, 2.1900e+00, 1.8800e+00, 2.2700e+00, 1.8300e

In [32]:
# Fixing in_IrrigatedAcreage datatype
outdf['in_IrrigatedAcreage'] = pd.to_numeric(outdf['in_IrrigatedAcreage'], errors='coerce').fillna(0)
outdf['in_IrrigatedAcreage'].unique()

array([  0.  ,  10.  ,  38.5 , ..., 658.5 ,  80.99, 144.07])

In [33]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['wadeID1', 'wadeID2', 'wadeID3', ..., 'wadeID18849', 'wadeID18850',
       'wadeID18851'], dtype=object)

## Shapefile Data
- For attaching gemetry to csv inputs.

In [34]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('shapefile/OR_PoU2.shp')
dfPoUshapetemp.head(3)

,pou_displa,pou_disp_1,wris_link,snp_id,pou_use_id,app_char,app_nbr,permit_cha,permit_nbr,cert_nbr,claim_char,claim_nbr,decree_tit,transfer_n,wr_type,name_last,name_first,name_compa,use_code,use_catego,use_code_d,priority_d,supplement,wris_acres,technician,agency,rec_creati,last_updt_,feature_qu,delta_size,remarks,Shape_Leng,Shape_Area,geometry
0,App: P 81441 * LV,P 81441,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,5135,4124,P,81441,None,0,0,None,0,None,None,ST,WILLIAMS,TERRY,DASH W BAR RANCH,LV,8,LIVESTOCK,1996-08-07,0,0.00000,MW,OWRD,2005-06-30,2005-07-22,30,0.00000,PLACED USING DRG,2654.80928,281203.13083,"POLYGON ((-118.36564 43.73765, -118.36428 43.7..."
1,App: P 82980 * ST,P 82980,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,6333,5886,P,82980,None,0,0,None,0,None,None,ST,PARKER,SOFIA,None,ST,M,STORAGE,1997-01-07,0,0.00000,BRW,OWRD,2016-04-11,2016-04-11,30,0.00000,PLACED USING 2014 IMAGERY,1437.65417,35294.34007,"POLYGON ((-123.04147 42.44192, -123.04133 42.4..."
2,Inchoate: T 4213 CF (REG) * IR,T 4213,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...,17008,7696,None,0,None,0,0,None,0,East Mud Creek,T 4213,SW,RENCKEN,DONALD,None,IR,3,IRRIGATION,1892-12-31,0,10.00000,MIGRT,OWRD,1997-12-01,1997-12-01,0,0.00000,0 CD 31,2426.48477,379563.11666,"POLYGON ((-118.43940 45.99213, -118.44205 45.9..."


In [35]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['pou_use_id'].replace("", 0).fillna(0).astype(int).astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

,in_SiteNativeID,geometry
0,POU4124,"POLYGON ((-118.36564 43.73765, -118.36428 43.7..."
1,POU5886,"POLYGON ((-123.04147 42.44192, -123.04133 42.4..."
2,POU7696,"POLYGON ((-118.43940 45.99213, -118.44205 45.9..."


## The Output

In [36]:
outdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462170 entries, 0 to 462169
Data columns (total 63 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   WaDEUUID                                      462170 non-null  object        
 1   in_MethodUUID                                 462170 non-null  object        
 2   in_VariableSpecificUUID                       462170 non-null  object        
 3   in_OrganizationUUID                           462170 non-null  object        
 4   in_Geometry                                   462170 non-null  object        
 5   in_GNISFeatureNameCV                          462170 non-null  object        
 6   in_WaterQualityIndicatorCV                    462170 non-null  object        
 7   in_WaterSourceName                            462170 non-null  object        
 8   in_WaterSourceNativeID                        462170 n

In [37]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,orD0,ORwr_M1,ORwr_V1,ORwr_O1,,,,FORMOSA 1 ADIT ...,wadeID1,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.85581,-123.38288,,,POD,21755_1,POD6909,,well,OR,,,,,,,,0.00000,,0.04000,WaDE Unspecified,21755,Formosa Exploration Inc,1989-11-21,,12/31,1/1,,0.00000,MINING,,,,,,0,,0.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
1,orD1,ORwr_M1,ORwr_V1,ORwr_O1,,,,SILVER BUTTE 1 ADIT ...,wadeID2,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.85455,-123.38349,,,POD,21755_2,POD6910,,well,OR,,,,,,,,0.00000,,0.00500,WaDE Unspecified,21755,Formosa Exploration Inc,1989-11-21,,12/31,1/1,,0.00000,MINING,,,,,,0,,0.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
2,orD2,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL ...,wadeID3,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.68227,-123.62942,,,POD,23327_1,POD9355,,well,OR,,,,,,,,0.00000,,0.11000,WaDE Unspecified,23327,Dutch Mining Llc,1994-05-06,,12/31,1/1,,0.00000,MINING,,,,,,0,,0.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
3,orD3,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL,wadeID4,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,44.30104,-121.65463,,,POD,23390_1,POD9480,,well,OR,,,,,,,,0.00000,,0.67000,WaDE Unspecified,23390,Jeff Steyaert Knife River Corp,1995-01-18,,12/31,1/1,,0.00000,MINING,,,,,,0,,0.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
4,orD4,ORwr_M1,ORwr_V1,ORwr_O1,,,,A WELL,wadeID4,Groundwater,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,44.30104,-121.65463,,,POD,23390_1,POD9480,,well,OR,,,,,,,,0.00000,,0.11000,WaDE Unspecified,23390,Jeff Steyaert Knife River Corp,1995-01-18,,12/31,1/1,,0.00000,MINING,,,,,,0,,0.00000,,,,,,,,http://apps.wrd.state.or.us/apps/wr/wrinfo/wr_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462165,orU109571,ORwr_M1,ORwr_V1,ORwr_O1,,,,WaDE Unspecified,wadeID18851,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.00062,-121.51901,,,POU,WaDE Unspecified,POU269184,,WaDE Unspecified,OR,,,,,,,,0.00000,,0.00000,WaDE Unspecified,,WaDE Unspecified,NaT,,,,,0.00000,WaDE Unspecified,,,,,,0,,28.50000,,,,,,,,
462166,orU109572,ORwr_M1,ORwr_V1,ORwr_O1,,,,WaDE Unspecified,wadeID18851,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,WaDE Unspecified,4326,,,,42.09869,-121.70515,,,POU,WaDE Unspecified,POU269187,,WaDE Unspecified,OR,,,,,,,,0.00000,,0

In [38]:
# Export the output dataframe
outdf.to_csv('Pwr_orMain.zip', index=False, compression="zip")  # The output, save as a zip
dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.